In [1]:
MARKET = 'NSE'

In [2]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [3]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder
    BAR_FORMAT = "{l_bar}{bar:-20}{r_bar}"

In [4]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = cwd.joinpath(THIS_FOLDER, 'data', 'log', 'temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [5]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

In [35]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
np.sort(np.array(files))

array(['df_chains.pkl', 'df_nakeds.pkl', 'df_ohlcs.pkl',
       'df_opt_prices.pkl', 'df_symlots.pkl', 'df_unds.pkl', 'dfrq.pkl',
       'qopt_rejects.pkl', 'qopts.pkl'], dtype='<U17')

In [38]:
cols = ['conId', 'symbol', 'expiry', 'strike', 'secType', 'dte', 'right',
       'contract', 'und_iv', 'undPrice', 'hi_sd', 'lo_sd', 'lot', 'comm',
       'margin', 'bid', 'ask', 'close', 'last', 'sdMult', 'iv', 'qty',
       'intrinsic', 'timevalue', 'rom', 'expRom', 'price', 'expPrice']
df_nakeds[cols]

,conId,symbol,expiry,strike,secType,dte,right,contract,und_iv,undPrice,hi_sd,lo_sd,lot,comm,margin,bid,ask,close,last,sdMult,iv,qty,intrinsic,timevalue,rom,expRom,price,expPrice
0,457599185,SRTRANSFI,20201231,1340.0,OPT,29,C,"Option(conId=457599185, symbol='SRTRANSFI', la...",0.547238,1023.55,1307.741124,739.358876,667.0,20.0,166691.44,-1.0,-1.0,22.35,NaN,2.004320,0.547238,667.0,0.0,22.35,1.124092,1.124092,22.35,22.35
1,457995488,CHOLAFIN,20201231,490.0,OPT,29,C,"Option(conId=457995488, symbol='CHOLAFIN', las...",0.480371,374.80,466.148569,283.451431,2500.0,20.0,233966.02,NaN,NaN,7.85,NaN,2.269986,0.480371,2500.0,0.0,7.85,1.054651,1.054651,7.85,7.85
2,457996865,SRTRANSFI,20201231,1360.0,OPT,29,C,"Option(conId=457996865, symbol='SRTRANSFI', la...",0.547238,1023.55,1307.741124,739.358876,667.0,20.0,160246.54,NaN,NaN,19.80,NaN,2.130995,0.547238,667.0,0.0,19.80,1.035712,1.035712,19.80,19.80
3,457996099,BAJFINANC,20201231,6300.0,OPT,29,C,"Option(conId=457996099, symbol='BAJFINANC', la...",0.423827,4861.10,5906.418084,3815.781916,250.0,20.0,225023.67,NaN,NaN,54.20,NaN,2.477734,0.423827,250.0,0.0,54.20,0.756771,1.000000,54.20,71.60
4,457242932,MMFIN,20201231,225.0,OPT,29,C,"Option(conId=457242932, symbol='MMFIN', lastTr...",0.535010,167.70,213.221873,122.178127,3444.0,20.0,128167.29,-1.0,-1.0,2.20,NaN,2.265724,0.535010,3444.0,0.0,2.20,0.742088,1.000000,2.20,2.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,447280562,PEL,20201231,650.0,OPT,29,P,"Option(conId=447280562, symbol='PEL', lastTrad...",0.508151,1414.45,1779.124667,1049.775333,550.0,20.0,445412.78,-1.0,-1.0,0.05,NaN,3.773254,0.508151,550.0,0.0,0.05,0.000212,1.000000,0.05,235.95
379,453309559,ICICIGI,20201231,700.0,OPT,29,P,"Option(conId=453309559, symbol='ICICIGI', last...",0.390073,1471.15,1762.307469,1179.992531,425.0,20.0,100058.05,-1.0,-1.0,0.05,NaN,4.767420,0.390073,425.0,0.0,0.05,0.000157,1.000000,0.05,318.00
380,452789713,ICICIGI,20201231,720.0,OPT,29,P,"Option(conId=452789713, symbol='ICICIGI', last...",0.390073,1471.15,1762.307469,1179.992531,425.0,20.0,100058.05,-1.0,-1.0,0.05,NaN,4.643776,0.390073,425.0,0.0,0.05,0.000157,1.000000,0.05,318.00
381,447327703,PVR,20201231,550.0,OPT,29,P,"Option(conId=447327703, symbol='PVR', lastTrad...",0.511810,1322.15,1665.482483,978.817517,407.0,20.0,85527.35,NaN,NaN,0.05,NaN,4.048175,0.511810,407.0,0.0,0.05,0.000052,1.000000,0.05,970.75


In [41]:
from support import get_prob
get_prob(1.8)

0.9281393617741486